# 1. 크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re
import pickle

In [ ]:
service = Service('../chromedriver/chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [ ]:
pages = [] # 크롤링 오류 확인 위해 page 수 기록
title = []
lyric = []
artist = []
genres = []
writer = []
rank = []
p = re.compile("'(\d+)'")
for page in range(9201,10001,50): # 1페이지만 추출(1,51,50), 발라드 gnrCode=GN0100, 댄스 gnrCode=GN0200
    url = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0200#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D\
                =&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex={}".format(page) # 댄스 페이지
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 한 페이지에 있는 50개의 songId 추출
    songid = [] # songid

    for i in range(50):
        data = soup.find_all("a", class_= "btn button_icons type03 song_info")[i].get('href')
        id_ = p.search(str(data)).group(1)
        songid.append(id_)
        f = soup.select('td span.rank')[i].text
        rank.append(f)

    # 해당 songId 입력 후 정보 추출
    for song in songid:
        lyric_url = 'https://www.melon.com/song/detail.htm?songId={}'.format(song)
        driver.get(lyric_url)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        a = soup.select('div.song_name')[0].text[3:].strip() # 제목
        title.append(a)
        try:
            driver.find_element_by_css_selector(".button_more.arrow_d").click() # 가사
            ly = driver.find_element_by_css_selector('.lyric.on')
            b = ly.text
            lyric.append(b)
        except:
            lyric.append('가사없음')
        ar = driver.find_element_by_css_selector(".artist")
        c = ar.text
        artist.append(c)
        d = soup.select('dl.list dd')[2].text # 장르
        genres.append(d)
        try:
            e = soup.select('div.section_prdcr a.artist_name')[0].text # 작사가 1명
            writer.append(e)
        except:
            writer.append('작사가없음')
        pages.append(page)

<ipython-input-136-889516fe7638>:38: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector(".button_more.arrow_d").click() # 가사
<ipython-input-136-889516fe7638>:39: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ly = driver.find_element_by_css_selector('.lyric.on')
<ipython-input-136-889516fe7638>:44: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ar = driver.find_element_by_css_selector(".artist")


In [ ]:
col = ['pages','rank','title','lyric','artist','genres','writer']
songlist = pd.DataFrame([pages, rank,title, lyric, artist, genres, writer]).T
songlist.columns = col
songlist

,pages,rank,title,lyric,artist,genres,writer
0,9201,9201,"Trouble (Prod. by 용준형, 김태주)",왜 종일 신경이 네게 꽂혀있는지\n나 요즘 이상해 뭐에 홀린 것 같이\n스쳐 지나갔...,크리샤 츄 (Kriesha Chu),댄스,Good Life
1,9201,9202,파라다이스 (Limited Ver.),Daylight 너무나 눈부셔\n두 발 어디론가 내딛어\nToday 떠나기 좋은 날...,DIA (다이아),댄스,STAINBOYS
2,9201,9203,허망로맨스,너와 함께 하던 순간\n나는 점점 사랑을 느꼈어\n우리 사이는 어느 순간 친구 이상...,쵸쵸우,댄스,뽕필학생
3,9201,9204,24/7,매일매일 들리는 투정들\n일주일을 반복해도 끝이 없는걸\nEveryday yeah\...,TWICE (트와이스),댄스,나연
4,9201,9205,Magic Dream,콩닥콩닥 내 심장이 떨려오는 걸\n부끄럽게 얼굴이 빨개\n자꾸만 빨개져\n어머 어머...,에이프릴 (APRIL),"댄스, 국내드라마",한준
...,...,...,...,...,...,...,...
95,None,9296,None,None,None,None,None
96,None,9297,None,None,None,None,None
97,None,9298,None,None,None,None,None
98,None,9299,None,None,None,None,None


In [ ]:
songlist = songlist.head(50)
songlist

,pages,rank,title,lyric,artist,genres,writer
0,9201,9201,"Trouble (Prod. by 용준형, 김태주)",왜 종일 신경이 네게 꽂혀있는지\n나 요즘 이상해 뭐에 홀린 것 같이\n스쳐 지나갔...,크리샤 츄 (Kriesha Chu),댄스,Good Life
1,9201,9202,파라다이스 (Limited Ver.),Daylight 너무나 눈부셔\n두 발 어디론가 내딛어\nToday 떠나기 좋은 날...,DIA (다이아),댄스,STAINBOYS
2,9201,9203,허망로맨스,너와 함께 하던 순간\n나는 점점 사랑을 느꼈어\n우리 사이는 어느 순간 친구 이상...,쵸쵸우,댄스,뽕필학생
3,9201,9204,24/7,매일매일 들리는 투정들\n일주일을 반복해도 끝이 없는걸\nEveryday yeah\...,TWICE (트와이스),댄스,나연
4,9201,9205,Magic Dream,콩닥콩닥 내 심장이 떨려오는 걸\n부끄럽게 얼굴이 빨개\n자꾸만 빨개져\n어머 어머...,에이프릴 (APRIL),"댄스, 국내드라마",한준
5,9201,9206,Katchup,질질 끌려다녔어\n절절한 맘잡고\n일일이 챙겨주고\n얼얼한 채\n목에 탁 걸려 으흠...,달수빈,댄스,달수빈
6,9201,9207,좋아해 (Prod. Flow Blow),오늘밤 아닌 all day\n생각만해도 설레\n보기도 전인 건데\n기분은 춤추는 홍...,JBJ95,댄스,iHwak
7,9201,9208,What Do You Think,잘 생각해 다른 놈 맘 흔든 적 있어\n잘 봐 이제 너 나한테 흔들릴 거라고\n좋아...,틴탑,댄스,ChangJo
8,9201,9209,여름안에서 (IN SUMMER),난 너를 사랑해 난 너를 사랑해\n난 너를 사랑해 난 너를 사랑해\n언제나 꿈꿔 온...,멋진녀석들 (GreatGuys),댄스,이현도
9,9201,9210,떠나,어제 네 목소리 한숨이 가득해\n차를 타고 어디론가 떠나려고 해\n비가 내리고 있음...,앤드뉴 (Andnew),댄스,앤드뉴 (Andnew)


In [ ]:
songlist.to_pickle('dance_9201-9250.pkl', protocol = 4) # 발라드는 ballad로 저장

In [ ]:
songlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   pages   100 non-null    object
 1   title   100 non-null    object
 2   lyric   100 non-null    object
 3   artist  100 non-null    object
 4   genres  100 non-null    object
 5   writer  100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB


In [ ]:
col = ['pages','rank','title','lyric','artist','genres','writer']
songlist = pd.DataFrame([pages, rank,title, lyric, artist, genres, writer]).T
songlist.columns = col
songlist

In [ ]:
data1 = pd.read_pickle('dance_10001-13550.pkl')
data2 = pd.read_pickle('dance_13551-13750.pkl')
data3 = pd.read_pickle('dance_13751-14100.pkl')
data4 = pd.read_pickle('dance_14101-14300.pkl')
data5 = pd.read_pickle('dance_14301-14350.pkl')

In [ ]:
df = pd.concat([data1, data2,data3,data4, data5 ],ignore_index=True)
#df = df.drop_duplicates()
df.to_pickle('dance_10001-14350.pkl', protocol = 4)